In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:
data = pd.read_csv('../openreview.csv')
binary_decisions = {'Accept (Oral)':1, 'Accept (Poster)':1, 'Accept (Spotlight)':1, 'Accept (Talk)':1, 'Withdrawn':0, 'Reject':0, 'Invite to Workshop Track':0}

In [23]:
def logistic_regression(cat):
    scores =[]
    theorem_indicator=[]
    target=[]

    for rating, decision, theorem, category in zip(data['ratings'], data['decisions'], data['theorem'], data['categories_list']):
        if theorem == 'pdf miss':
            continue
        try:
            category_list = category.split(';')
            found = False
            for c in category_list: 
                if int(c)==cat:
                    found=True
                    break
            if not(found): continue
        except: continue

        if theorem =='y':
            theorem_indicator.append(1)
        else:
            theorem_indicator.append(0)

        avg_rating=0
        for i in rating.split(';'):
            avg_rating += int(i)
        scores.append(avg_rating/len(rating.split(';')))

        target.append(binary_decisions[decision])

    x=pd.DataFrame()
    x['mean reviewer score']=scores
    x['theorem']=theorem_indicator
    x['constant']=[1]*len(scores)
    y=np.array(target)

    logit_model = sm.Logit(y, x)
    summary = logit_model.fit().summary()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=0)
    logisticRegr = LogisticRegression(random_state=0)
    model = logisticRegr.fit(x_train, y_train)
    accuracy = logisticRegr.score(x_test, y_test)

    return summary, accuracy

In [46]:
summary, score = logistic_regression(11)
print(summary)
print('Accuracy: ', score)

Optimization terminated successfully.
         Current function value: 0.591442
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  308
Model:                          Logit   Df Residuals:                      306
Method:                           MLE   Df Model:                            1
Date:                Sat, 02 Oct 2021   Pseudo R-squ.:                0.006428
Time:                        21:13:16   Log-Likelihood:                -182.16
converged:                       True   LL-Null:                       -183.34
Covariance Type:            nonrobust   LLR p-value:                    0.1247
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theorem        0.4187      0.271      1.547      0.122      -0.112       0.949
constant      -1.0622      0.